# ClipCard Agent Test: Civic/ICS
**Purpose:** This notebook tests whether an AI agent (LLM) can correctly parse, validate, and reason about ClipCards in the Civic / Incident Command System (ICS) domain.
It evaluates:1. Can the agent extract kill criteria from an ICS ClipCard?2. Can it determine if a given field report (scenario) would trigger the criteria?3. Can it suggest appropriate authority windows for different operational risks?

In [ ]:
import jsonimport pandas as pd# Load example ClipCardtry:    with open('../../examples/ics-test.clipcard.json', 'r') as f:        test_card = json.load(f)    print("Loaded test ClipCard:")    print(json.dumps(test_card, indent=2))except FileNotFoundError:    print("No test ClipCard found.")    test_card = None

### Test 1: Extract Kill Criteria
Can we programmatically parse the demobilization triggers (kill criteria)?

In [ ]:
if test_card:    kill_criteria = test_card.get('kill_criteria', [])    print(f"Found {len(kill_criteria)} kill criteria:\n")    for i, criterion in enumerate(kill_criteria, 1):        print(f"{i}. Condition: {criterion['condition']}")        print(f"   Action: {criterion['action']}\
")else:    print("No ClipCard available for testing.")

### Test 2: Scenario Evaluation
Test if field reports (scenarios) from an operational period would trigger the kill criteria.

In [ ]:
# Define test scenarios for ICS operationsscenarios = [    {        "name": "Normal Operations",        "median_queue_time_minutes": 15,        "water_stockout": false,        "safety_incident_class": 0,        "expected_trigger": false    },    {        "name": "Long Queues",        "median_queue_time_minutes": 40,        "water_stockout": false,        "safety_incident_class": 0,        "expected_trigger": true    },    {        "name": "Logistics Failure",        "median_queue_time_minutes": 20,        "water_stockout": true,        "safety_incident_class": 1,        "expected_trigger": true    },    {        "name": "Safety Incident",        "median_queue_time_minutes": 10,        "water_stockout": false,        "safety_incident_class": 3,        "expected_trigger": true    }],def evaluate_ics_scenarios(card, scenarios):    results = []    criteria = card.get('kill_criteria', [])        for scenario in scenarios:        triggered = False        for rule in criteria:            condition = rule['condition']            if 'median_queue_time_minutes' in condition and scenario['median_queue_time_minutes'] > 30:                triggered = True            if 'water_stockout' in condition and scenario['water_stockout'] == True:                triggered = True            if 'safety_incident_class' in condition and scenario['safety_incident_class'] >= 2:                triggered = True                match = triggered == scenario['expected_trigger']        results.append({            'Scenario': scenario['name'],            'Triggered': triggered,            'Expected': scenario['expected_trigger'],            'Match': '✓' if match else '✗'        })    return pd.DataFrame(results)if test_card:    df_results = evaluate_ics_scenarios(test_card, scenarios)    print("Evaluating ICS scenarios against the test ClipCard:\n")    print(df_results.to_string(index=False))    print(f"\nTest Accuracy: {df_results['Match'].value_counts().get('✓', 0)}/{len(df_results)}")

### Test 3: Authority Window Recommendations
Generate appropriate authority windows for an operational plan based on risk factors.

In [ ]:
def recommend_ics_authority_window(impact, uncertainty, reversibility):    """Recommends an ICS-specific authority window based on risk."""    risk_score = impact * uncertainty        if risk_score >= 20 or reversibility >= 5:        return {            "scope_limit": "Single Group or Task Force only",            "time_limit": "4 hours (re-evaluate at mid-OP)",            "auto_pause": true,            "justification": "Very high risk requires tight control and frequent re-evaluation."        }    elif risk_score >= 15 or reversibility >= 4:        return {            "scope_limit": "Single Division, limited resources",            "time_limit": "1 Operational Period (12h)",            "auto_pause": true,            "justification": "High risk requires containing the action within one Division and OP."        }    else:        return {            "scope_limit": "Branch-level, multiple Divisions",            "time_limit": "2-3 Operational Periods",            "auto_pause": false,            "justification": "Lower risk allows for broader, multi-day operations."        }# Test with example ICS scenariostest_cases = [    {"name": "New evacuation route", "impact": 5, "uncertainty": 5, "reversibility": 4},    {"name": "Deploying volunteer-led traffic control", "impact": 3, "uncertainty": 4, "reversibility": 2},    {"name": "Opening a new supply point", "impact": 2, "uncertainty": 3, "reversibility": 1}],print("ICS Authority Window Recommendations:\n")for case in test_cases:    rec = recommend_ics_authority_window(case['impact'], case['uncertainty'], case['reversibility'])    print(f"{case['name']} (I={case['impact']}, U={case['uncertainty']}, R={case['reversibility']}):")    print(f"  Scope: {rec['scope_limit']}")    print(f"  Duration: {rec['time_limit']}")    print(f"  Auto-pause: {rec['auto_pause']}")    print(f"  Reason: {rec['justification']}\
")